This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [4]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'HztRSXX8XCj1jSycaqih'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [16]:
# First, import the relevant modules
import requests
import json

In [12]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/EON_X?start_date=2019-02-08&end_date=2019-02-08&api_key=" + API_KEY)

In [13]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.text

'{"dataset":{"id":36592406,"dataset_code":"EON_X","database_code":"FSE","name":"E.on Se (EON_X)","description":"Stock Prices for E.on Se (EON) from the Frankfurt Stock Exchange.\\u003cbr\\u003e\\u003cbr\\u003eTrading System: Xetra\\u003cbr\\u003e\\u003cbr\\u003eISIN: DE000ENAG999","refreshed_at":"2019-02-08T22:30:21.034Z","newest_available_date":"2019-02-08","oldest_available_date":"2003-01-20","column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"frequency":"daily","type":"Time Series","premium":false,"limit":null,"transform":null,"column_index":null,"start_date":"2019-02-08","end_date":"2019-02-08","data":[["2019-02-08",null,9.524,9.409,9.45,null,null,null,null,null,null]],"collapse":null,"order":null,"database_id":6129}}'

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [85]:
r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/EON_X?start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY)

Convert the returned JSON object into a Python dictionary.

In [86]:
data = json.loads(r.text)

In [87]:
dataset = data['dataset']
stock_data = dataset['data']

Calculate what the highest and lowest opening prices were for the stock in this period.

In [88]:
# Define a dictionary to translate column names into correspondind index values.
# Use the dictionary to define a function that will extract the desired columns from the list of lists.
dict_key = dict(zip(dataset['column_names'], range(0,len(dataset['column_names']))))
def get_column(column_name):
    return_column = []
    
    for row in stock_data:
        return_column.append(row[dict_key[column_name]])
        
    return return_column

In [101]:
max_open = max([entry for entry in get_column('Open') if entry is not None])
low_open = min([entry for entry in get_column('Open') if entry is not None])
max_open

10.75

In [102]:
low_open

6.65

What was the largest change in any one day (based on High and Low price)?

In [95]:
maximum_change = max([entry_high - entry_low for (entry_low, entry_high) in list(zip(get_column('Low'), get_column('High'))) if entry_low is not None and entry_high is not None])
maximum_change

0.6999999999999993

What was the largest change between any two days (based on Closing Price)?

In [96]:
# Create a list of positive differences.  Obtain the maximum value from the resulting list.
consecutive_difference = [get_column('Close')[index+1] - get_column('Close')[index] for index in range(0, len(get_column('Close'))-1)]
max_difference = max([each if each>0 else -each for each in consecutive_difference])
max_difference

0.47999999999999865

What was the average daily trading volume during this year?

In [97]:
# Create a list of non-null volumes.  Calculate the averate from the obtained list.
not_none_volume = [each for each in get_column('Traded Volume') if each is not None]
average_volume = sum(not_none_volume)/len(not_none_volume)
average_volume

11708661.142857144

(Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [98]:
# Define a function to determine the arithmetic median of a discrete series.
def median_of_list(arg_list):
    
    if len(arg_list)%2 == 0:
        median = (arg_list[int(len(arg_list)/2) -1] + arg_list[int(len(arg_list)/2)])/2
        
    else:
        median = arg_list[int((len(arg_list) - 1)/2)]
    
    return median

In [99]:
# Use the function to obtain the median of the non-null volumes.
median_volume = median_of_list(not_none_volume)
median_volume

10544312.5